In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

pd.pandas.set_option('display.max_columns', None)

### Data Loading

In [19]:
data=pd.read_csv('./Train/X_Scaled_train.csv')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   SalePrice      1460 non-null   float64
 2   MSSubClass     1460 non-null   float64
 3   MSZoning       1460 non-null   float64
 4   LotFrontage    1460 non-null   float64
 5   LotArea        1460 non-null   float64
 6   Street         1460 non-null   float64
 7   Alley          1460 non-null   float64
 8   LotShape       1460 non-null   float64
 9   LandContour    1460 non-null   float64
 10  LotConfig      1460 non-null   float64
 11  LandSlope      1460 non-null   float64
 12  Neighborhood   1460 non-null   float64
 13  Condition1     1460 non-null   float64
 14  Condition2     1460 non-null   float64
 15  BldgType       1460 non-null   float64
 16  HouseStyle     1460 non-null   float64
 17  OverallQual    1460 non-null   float64
 18  OverallC

In [21]:
data.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,12.247694,0.235294,0.5,0.0,0.366344,0.0,0.5,1.0,1.0,0.75,0.0,0.181818,0.4,0.0,0.0,0.4,0.666667,0.500,0.036765,0.098361,0.0,0.0,0.8,0.8,0.25,0.12250,0.333333,1.0,0.50,0.5,1.00,1.00,0.333333,0.125089,1.0,0.0,0.064212,0.140098,0.0,0.0,1.0,1.0,0.356155,0.413559,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.6,0.0,0.002488,0.666667,0.50,0.386460,1.0,1.0,1.0,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.090909,0.50,1.0,0.5
1,2,12.109011,0.000000,0.5,0.0,0.391317,0.0,0.5,1.0,1.0,0.50,0.0,0.727273,0.2,0.0,0.0,0.2,0.555556,0.875,0.227941,0.524590,0.0,0.0,0.4,0.4,0.50,0.00000,1.000000,1.0,0.25,0.5,1.00,0.25,0.000000,0.173281,1.0,0.0,0.121575,0.206547,0.0,0.0,1.0,1.0,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,1.000000,0.333333,1.0,0.333333,1.0,0.0,0.015423,0.666667,0.50,0.324401,1.0,1.0,1.0,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.363636,0.25,1.0,0.5
2,3,12.317167,0.235294,0.5,0.0,0.422359,0.0,0.5,0.0,1.0,0.75,0.0,0.181818,0.4,0.0,0.0,0.4,0.666667,0.500,0.051471,0.114754,0.0,0.0,0.8,0.8,0.25,0.10125,0.333333,1.0,0.50,0.5,1.00,0.75,0.333333,0.086109,1.0,0.0,0.185788,0.150573,0.0,0.0,1.0,1.0,0.383441,0.419370,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,1.0,0.0,0.003483,0.666667,0.50,0.428773,1.0,1.0,1.0,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.727273,0.50,1.0,0.5
3,4,11.849398,0.294118,0.5,0.0,0.390295,0.0,0.5,0.0,1.0,0.00,0.0,0.227273,0.4,0.0,0.0,0.4,0.666667,0.500,0.669118,0.606557,0.0,0.0,0.9,1.0,0.50,0.00000,1.000000,1.0,0.00,1.0,0.25,1.00,0.000000,0.038271,1.0,0.0,0.231164,0.123732,0.0,0.5,1.0,1.0,0.399941,0.366102,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.4,0.6,0.003980,1.000000,0.75,0.452750,1.0,1.0,1.0,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.090909,0.00,1.0,0.0
4,5,12.429216,0.235294,0.5,0.0,0.468761,0.0,0.5,0.0,1.0,0.50,0.0,0.590909,0.4,0.0,0.0,0.4,0.777778,0.500,0.058824,0.147541,0.0,0.0,0.8,0.8,0.25,0.21875,0.333333,1.0,0.50,0.5,1.00,0.00,0.333333,0.116052,1.0,0.0,0.209760,0.187398,0.0,0.0,1.0,1.0,0.466237,0.509927,0.0,0.666523,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,1.0,0.0,0.003980,0.666667,0.75,0.589563,1.0,1.0,1.0,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,1.000000,0.50,1.0,0.5


In [22]:
data.tail()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1455,1456,12.072541,0.235294,0.5,0.0,0.353592,0.0,0.5,1.0,1.0,0.75,0.0,0.318182,0.4,0.0,0.0,0.4,0.555556,0.500,0.058824,0.131148,0.0,0.0,0.8,0.8,0.5,0.000000,1.000000,1.000000,0.50,0.5,1.00,1.00,1.000000,0.000000,1.000000,0.000000,0.407962,0.155974,0.0,0.0,1.0,1.0,0.396777,0.336077,0.0,0.564433,0.000000,0.0,0.666667,0.5,0.375,0.333333,1.000000,0.416667,1.0,0.333333,1.0,0.0,0.003980,0.666667,0.50,0.324401,1.0,1.0,1.0,0.000000,0.073126,0.000000,0.0,0.0,0.0,0.0,0.50,0.0,0.00000,0.636364,0.25,1.0,0.5
1456,1457,12.254863,0.000000,0.5,0.0,0.453273,0.0,0.5,1.0,1.0,0.75,0.0,0.545455,0.4,0.0,0.0,0.2,0.555556,0.625,0.235294,0.377049,0.0,0.0,0.5,0.5,1.0,0.074375,1.000000,1.000000,0.25,0.5,1.00,1.00,0.000000,0.139972,0.833333,0.110583,0.252140,0.252373,0.0,1.0,1.0,1.0,0.690872,0.000000,0.0,0.645810,0.333333,0.0,0.666667,0.0,0.375,0.333333,1.000000,0.416667,0.0,0.666667,1.0,0.0,0.015920,1.000000,0.50,0.352609,1.0,1.0,1.0,0.407235,0.000000,0.000000,0.0,0.0,0.0,0.0,0.75,0.0,0.00000,0.090909,1.00,1.0,0.5
1457,1458,12.493130,0.294118,0.5,0.0,0.379597,0.0,0.5,1.0,1.0,0.75,0.0,0.227273,0.4,0.0,0.0,0.4,0.666667,1.000,0.507353,0.081967,0.0,0.0,0.2,0.2,0.5,0.000000,0.000000,0.333333,0.75,1.0,0.25,1.00,0.333333,0.048724,1.000000,0.000000,0.375428,0.188543,0.0,0.0,1.0,1.0,0.480189,0.557869,0.0,0.688669,0.000000,0.0,0.666667,0.0,0.500,0.333333,0.666667,0.583333,1.0,0.666667,0.4,0.0,0.034328,0.666667,0.25,0.177715,1.0,1.0,1.0,0.000000,0.109689,0.000000,0.0,0.0,0.0,0.0,0.00,1.0,0.16129,0.363636,1.00,1.0,0.5
1458,1459,11.864462,0.000000,0.5,0.0,0.393688,0.0,0.5,1.0,1.0,0.75,0.0,0.500000,0.4,0.0,0.0,0.2,0.444444,0.625,0.441176,0.245902,0.5,0.0,0.4,0.4,0.5,0.000000,1.000000,1.000000,0.25,1.0,1.00,0.75,0.333333,0.008682,0.833333,0.698100,0.000000,0.176432,0.0,0.5,1.0,0.0,0.443419,0.000000,0.0,0.414497,0.333333,0.0,0.333333,0.0,0.250,0.333333,0.666667,0.250000,1.0,0.000000,0.6,0.0,0.029851,1.000000,0.25,0.169252,1.0,1.0,1.0,0.427071,0.000000,0.202899,0.0,0.0,0.0,0.0,0.50,0.0,0.00000,0.272727,1.00,1.0,0.5
1459,1460,11.901583,0.000000,0.5,0.0,0.398069,0.0,0.5,1.0,1.0,0.75,0.0,0.272727,0.4,0.0,0.0,0.2,0.444444,0.625,0.316176,0.721311,0.0,0.0,0.3,0.3,0.5,0.000000,0.333333,1.000000,0.25,1.0,1.00,1.00,0.166667,0.147059,0.333333,0.196744,0.058219,0.205565,0.0,0.5,1.0,1.0,0.501253,0.000000,0.0,0.468559,0.333333,0.0,0.333333,0.5,0.375,0.333333,1.000000,0.333333,1.0,0.000000,0.6,0.0,0.021393,0.000000,0.25,0.194640,1.0,1.0,1.0,0.858810,0.124314,0.000000,0.0,0.0,0.0,0.0,0.50,0.0,0.00000,0.454545,0.50,1.0,0.5


## Feature Selection

``` creating y_train and x_train ```

``` Performing Lasso Regression to remove/penalize the higher slopes ```

``` Selecting features with SelectFromModel ```

In [23]:
## Target Variable
y_train=data[['SalePrice']]

In [24]:
## Features to train
X_train=data.drop(['Id','SalePrice'],axis=1)

In [25]:
## Model
feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=42))
feature_sel_model.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=42))

In [26]:
## getting model supported features, returns an array of supported features, True/False 
feature_sel_model.get_support()

array([False,  True, False, False, False, False,  True, False, False,
       False, False, False, False,  True, False,  True, False, False,
        True, False, False, False, False, False, False,  True, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False,  True,  True, False,  True, False, False,  True,
        True, False, False, False, False, False,  True, False, False,
        True, False,  True, False,  True,  True, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [27]:
selected_features = X_train.columns[(feature_sel_model.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_features)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(feature_sel_model.estimator_.coef_ == 0)))

total features: 78
selected features: 20
features with coefficients shrank to zero: 58


In [28]:
selected_features

Index(['MSZoning', 'LotShape', 'BldgType', 'OverallQual', 'YearRemodAdd',
       'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
       'CentralAir', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'KitchenQual',
       'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCars', 'PavedDrive'],
      dtype='object')

In [29]:
X_train=X_train[selected_features]

In [30]:
X_train.head()

,MSZoning,LotShape,BldgType,OverallQual,YearRemodAdd,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,GarageType,GarageFinish,GarageCars,PavedDrive
0,0.5,1.0,0.0,0.666667,0.098361,0.333333,0.5,1.00,0.333333,0.0,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.0,0.666667,0.50,1.0
1,0.5,1.0,0.0,0.555556,0.524590,1.000000,0.5,0.25,0.000000,0.0,1.0,0.503056,0.470245,0.000000,1.000000,0.333333,0.0,0.666667,0.50,1.0
2,0.5,0.0,0.0,0.666667,0.114754,0.333333,0.5,0.75,0.333333,0.0,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.0,0.666667,0.50,1.0
3,0.5,0.0,0.0,0.666667,0.606557,1.000000,1.0,1.00,0.000000,0.5,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.6,1.000000,0.75,1.0
4,0.5,0.0,0.0,0.777778,0.147541,0.333333,0.5,0.00,0.333333,0.0,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.0,0.666667,0.75,1.0


In [31]:
## Final Features with Id and SalePrice
final_features = [feature for feature in data.columns if feature in selected_features or feature in ['Id', 'SalePrice']]
final_data = data[final_features]
print(final_features)

['Id', 'SalePrice', 'MSZoning', 'LotShape', 'BldgType', 'OverallQual', 'YearRemodAdd', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'KitchenQual', 'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCars', 'PavedDrive']


In [32]:
final_data.head()

,Id,SalePrice,MSZoning,LotShape,BldgType,OverallQual,YearRemodAdd,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,GarageType,GarageFinish,GarageCars,PavedDrive
0,1,12.247694,0.5,1.0,0.0,0.666667,0.098361,0.333333,0.5,1.00,0.333333,0.0,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.0,0.666667,0.50,1.0
1,2,12.109011,0.5,1.0,0.0,0.555556,0.524590,1.000000,0.5,0.25,0.000000,0.0,1.0,0.503056,0.470245,0.000000,1.000000,0.333333,0.0,0.666667,0.50,1.0
2,3,12.317167,0.5,0.0,0.0,0.666667,0.114754,0.333333,0.5,0.75,0.333333,0.0,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.0,0.666667,0.50,1.0
3,4,11.849398,0.5,0.0,0.0,0.666667,0.606557,1.000000,1.0,1.00,0.000000,0.5,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.6,1.000000,0.75,1.0
4,5,12.429216,0.5,0.0,0.0,0.777778,0.147541,0.333333,0.5,0.00,0.333333,0.0,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.0,0.666667,0.75,1.0


In [33]:
final_data.to_csv('./Train/X_scaled_selected_train.csv', index=False)

### Feature Selection on Test Data
``` only taking those features which are present in training set ```

In [34]:
final_data_test = pd.read_csv('./Test/X_Scaled_test.csv')
test_features = [feature for feature in data.columns if feature in selected_features or feature in ['Id']]
final_data_test = final_data_test[test_features]
final_data_test.to_csv('./Test/X_scaled_selected_test.csv', index=False)